<a href="https://colab.research.google.com/github/omidbazgirTTU/LLMs/blob/main/PPO_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proximal Policy Implementation (PPO)
## Author Omid Bazgir
## Thanks for the great tutorial by Ehsan Kamalinejad (EK)
### Linke to the tutorial on YouTube https://www.youtube.com/watch?v=3uvnoVjM8nY&list=PLb9xatikqn0fwsS-Le1mkyQ2uZzK8DeP1&index=3

### Basic implementation of PPO (this example is for cart pole program) which is a reinforcement learning (RL) algorithm with many applications including game design, development, large language models (LLMs). PPO is being used in LLMs as as finetuning technique through reinforcement learning with human feedback (RLHF).

### more details on the cart pole problem including the how to define the reward values, action space and so on is provided in the Gymnasium documentation https://www.gymlibrary.dev/environments/classic_control/cart_pole/

References introduced by EK:
- [EK's Video Lecture](https://www.youtube.com/watch?v=3uvnoVjM8nY) This is the lecture where we did a deep dive into the theory of PPO.
- [OpenAI PPO Repo](https://github.com/openai/baselines/blob/master/baselines/ppo2/runner.py) This is helpful as a reference for further implementations.
- [PPO Paper](https://arxiv.org/abs/1707.06347) This is the original paper that introduced PPO.
- [Sergey Levine UC Berkley CS285](http://rail.eecs.berkeley.edu/deeprlcourse/) This is a complete course in RL.
- [Pieter Abbeel mini-course](https://www.youtube.com/watch?v=2GwBez0D20A&list=PLwRJQ4m4UJjNymuBM9RdmB3Z9N5-0IlY0) This is a mini-course focusing on TRPO, PPO, DDPG and model free RL.
- [OpenAI Documentation on RL](https://spinningup.openai.com/en/latest/index.html) THis is OpenAI documentation on RL and parts of our code was borrowed from here.
- [labml.ai](https://nn.labml.ai/) This repo contains popular papers with their annotated PyTorch implementations.
- [cleanrl](https://github.com/vwxyzjn/cleanrl) This repo has clean implementations of RL algorithms and parts of our code was borrowed from here.



In [3]:
# installing dependencies
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu116
!pip install moviepy omegaconf matplotlib
!pip install gym==0.26.2
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests
!pip install gym[classic_control] gym[atari] gym[accept-rom-license] gym[other]

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=0ed14226324aa969d5478c439b9e8505803ed644ac021e6e4c9efd9430867018
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827621 sha256=9b7ccf2ba04bc2a825a5e509ed9bc1bf2655322d039dea4acb210e45b9d251c4
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.0.6 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.
  Cloning https://github.com/carlosluis/stable-baselines3 (to revision fix_tests) to /tmp/pip-req-buil

In [4]:
import time
import random
import numpy as np
import matplotlib.pylab as plt
plt.style.use('dark_background')
from tqdm.notebook import tqdm
from omegaconf import DictConfig

import gym

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.distributions.categorical import Categorical

from IPython.display import Video

In [6]:
# set up
seed = 7
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [9]:
configs = {
    # experiment arguments
    "exp_name": "cartpole",
    "gym_id": "CartPole-v1", # the id of from OpenAI gym
    # training arguments
    "learning_rate": 1e-3, # the learning rate of the optimizer
    "total_timesteps": 1000000, # total timesteps of the training
    "max_grad_norm": 0.5, # the maximum norm allowed for the gradient
    # PPO parameters
    "num_trajcts": 32, # N
    "max_trajects_length": 64, # T
    "gamma": 0.99, # gamma
    "gae_lambda":0.95, # lambda for the generalized advantage estimation
    "num_minibatches": 2, # number of mibibatches used in each gradient
    "update_epochs": 2, # number of full rollout storage creations
    "clip_epsilon": 0.2, # the surrogate clipping coefficient
    "ent_coef": 0.01, # entroy coefficient controlling the exploration factor C2
    "vf_coef": 0.5, # value function controlling value estimation importance C1
    # visualization and print parameters
    "num_returns_to_average": 3, # how many episodes to use for printing average return
    "num_episodes_to_average": 23, # how many episodes to use for smoothing of the return diagram
    }

# batch_size is the size of the flatten sequences when trajcts are flatten
configs['batch_size'] = int(configs['num_trajcts'] * configs['max_trajects_length'])
# number of samples used in each gradient
configs['minibatch_size'] = int(configs['batch_size'] // configs['num_minibatches'])

configs = DictConfig(configs)

run_name = f"{configs.gym_id}__{configs.exp_name}__{seed}__{int(time.time())}"

## ENV
`envs` us set of parallel environments each holding a random initiali `state` and accepts an `action` to change and return its new state.

In [10]:
# creating an env with random state
def make_env_func(gym_id, seed, idx, run_name, capture_video = False):
  def env_fun():
    env = gym.make(gym_id, render_mode = "rgb_gray")
    env = gym.wrappers.RecordEpisodeStatistics(env)
    if capture_video:
      # initiate the video capture if not already initiated
      if idx ==0:
        #wrapper to create the video of the performance
        env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
    env.action_space.seed(seed)
    env.observation_space.seed(seed)

    return env
  return env_fun

In [14]:
# create N (here is 32) parallel envs
envs = []
for i in range(configs.num_trajcts):
  envs.append(make_env_func(configs.gym_id, seed+i, i, run_name))
envs = gym.vector.SyncVectorEnv(envs)
envs

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:623: UserWarning: WARN: The environment is being initialised with mode (rgb_gray) that is not in the possible render_modes (['human', 'rgb_array']).
  logger.warn(


SyncVectorEnv(32)

## Model
A simple MLP (or fully connected layers FC) model that gets a state and has two methods:
* `agent.value_func(state)` gets a state and returns the estimated expected total feature rewards from that state $V_{\theta}(s)$.

* `agent.policy(state)` gets a state and returns next `action`, `log_prob` of actions, the `entropy` and `value`.

In [15]:
class FCBlock(nn.Module):
  """ a generic fully connected residual block with good set up"""
  def __init__(self, embed_dim, dropout = 0.2):
    super().__init__()
    self.block = nn.Sequential(
        nn.LayerNorm(embed_dim),
        nn.GELU(),
        nn.Linear(embed_dim, 4*embed_dim),
        nn.GELU(),
        nn.Linear(4*embed_dim, embed_dim),
        nn.Dropout(dropout)
    )

    def forward(self, x):
      return x + self.block(x)

  class Agent(nn.Module):
    """ an agent that creates actions and estimates values"""
    def __init__(self, env_observation_dim, action_space_dim, embed_dim = 64, num_blocks=2):
      super().__init__()
      # getting the observation and embed that into another space `embed_dim`
      self.embedding_layer = nn.Linear(env_observation_dim, embed_dim)
      # layers that are shared between policy head and value head
      # it not necessarily needed to have a shared layer, but here since that value and policy tasks are quite similar
      # we can use several shared layer to do multi-task learning
      self.shared_layers = nn.Sequential(*[FCBlock(embed_dim=embed_dim) for _ in range(num_blocks)])
      self.value_head = nn.Linear(embed_dim, 1)
      self.policy_head = nn.Linear(embed_dim, action_space_dim)
      # orthogonal initialization with a hi entropy for exploration at the start
      torch.nn.init.orthogonal_(self.policy_head.weight, 0.01)

    def value_func(self,state):
      hidden = self.shared_layers(self.embedding_layer(state))
      value = self.value_head(hidden)
      return value

    def policy(self, state, action=None):
      # plicy is supposed to create actions but here it takes actions as the input
      # this is for phase 2 of PPO where we want to analze the actions
      hidden = self.shared_layers(self.embedding_layer(state))
      logits = self.policy_head(hidden)
      # Pytoch categorical class for sampling and probability calcucation
      probs = Categorical(logits=logits)
      if action is None:
        action = probs.sample()
      return action, probs.log_prob(action), probs.entropy(), self.value_head(hidden)

## Generalized Advantage Estimation (GAE)

### $Advantage (s, a)$ calculates how better or worse the return of taking the action $a$ at the state $s$ is compared to expected return for all other actions in that state.

### we can approximate that with the below reverse formulas

$δ_{t} = r_{t} + γV(s_{t+1}) - V(s_{t})$

$\hat{A_{t}} = δ_{t} + γλ\hat{A}_{t+1}$


In [16]:
def gae(
    cur_observations,   # the current state when advantages will be calculated
    rewards,            # rewards collected from trajectories of shape [num_trajcts, max_trajcts_length]
    dones,              # binary marker of end of trajectories of shape [num_trajcts, max_trajcts_length]
    values,             # value estimates collected over trajectories of shape [num_trajcts, max_trajcts_length]
):

  advantages = torch.zeros((configs.num_trajcts, configs.max_trajects_length))
  last_advantage = 0

  # the value after the last step
  with torch.no_grad():
    last_value = agent.value_func(cur_observations).reshape(1,-1)

  # reverse recursive to calculate advantages based on the delta formula
  for t in reversed(range(configs.max_trajects_length)):
    # mask if episode completed after step t
    mask = 1.0 - dones[:,t] # --> if we are looking for those trajectories that were ended quicker we don't need to any further calculation so we use the variable mask
    last_value = last_value * mask
    last_advantage = last_advantage * mask
    delta = rewards[:,t] + configs.gamma * last_value - values[:,t]
    last_advantage = delta + configs.gamma * configs.gae_lambda * last_advantage
    advantages[:,t] = last_advantage
    last_value = values[:,t]

  advantages = advantages.to(device)
  returns = advantages + values

  return advantages, returns

## Creating Rollout stage

### Phase 1: rollout creation

1. Generate $N$ trajectories of length $T$ by $\pi_{θ_{old}}$
2. calculate $logits_{old}$ for the actions
3. calculate $V_{theta}$ along the trajectories
4. calculate advantage estimates $\hat{A}_{t}$
5. create a storage and add all items to it

In [ ]:
def create_rollout(
    envs,               # parallel envs creating trajectories
    cur_observations,   # starting observation of shape [num_trajcts, observation_dim]
    cur_done,           # current termination status of shape [num_trajcts,]
    all_returns         # a list to track returns
)
  """
  rollout phase: create parallel trajectories and store them in the rolout storage
  """

  # cache empty tensors to store the rollouts